In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.types import *

spark = SparkSession.builder.appName("FilterRank").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/03 20:32:01 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/05/03 20:32:01 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/05/03 20:32:02 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/05/03 20:32:02 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [2]:
articleSchema = StructType().add("_c0", "string").add("_c1", "double")
csv_df = spark \
    .readStream \
    .option("delimiter", "\t") \
    .schema(articleSchema) \
    .csv("gs://hw2_spark_bucket_ak47/q8-whole") 

In [3]:
filter_csv_df = csv_df.filter(csv_df._c1>0.5)

In [4]:
query = filter_csv_df \
    .writeStream \
    .outputMode("append") \
    .format("csv") \
    .option("path", "/out") \
    .option("checkpointLocation", "/out") \
    .start()

# query.awaitTermination()

22/05/03 20:32:13 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [6]:
files = spark.read.csv("/out/part*.csv")
files.coalesce(1).write.format("csv").option("delimiter","\t").mode("overwrite").save("/out/all")

22/05/03 20:33:19 WARN org.apache.spark.sql.execution.streaming.FileStreamSource: Listed 1 file(s) in 4373 ms
